In [1]:
import spotlight
annotations = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate',
    'Berlin is the capital and largest city of Germany',confidence=0.4,support = 20)
annotations

[{'URI': 'http://dbpedia.org/resource/Berlin',
  'support': 87107,
  'types': 'Wikidata:Q515,Wikidata:Q486972,Schema:Place,Schema:City,DBpedia:Settlement,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:City',
  'surfaceForm': 'Berlin',
  'offset': 0,
  'similarityScore': 0.9983656061097194,
  'percentageOfSecondRank': 0.0008228013211064525},
 {'URI': 'http://dbpedia.org/resource/City',
  'support': 22610,
  'types': '',
  'surfaceForm': 'city',
  'offset': 34,
  'similarityScore': 0.8720405806057635,
  'percentageOfSecondRank': 0.10547609042001196},
 {'URI': 'http://dbpedia.org/resource/Germany',
  'support': 222752,
  'types': 'Wikidata:Q6256,Schema:Place,Schema:Country,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:Country',
  'surfaceForm': 'Germany',
  'offset': 42,
  'similarityScore': 0.9959557085639706,
  'percentageOfSecondRank': 0.0024005027968948783}]

In [2]:
annotations[0]["URI"]

'http://dbpedia.org/resource/Berlin'

In [3]:
def getURIs(json):
    URIs = []
    for obj in json:
        URIs.append(obj["URI"])

    return URIs
URIs = getURIs(annotations)

In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
endpoint = "https://dbpedia.org/sparql"
def sparql_to_dataframe(endpoint, query):
    """
    Convert SPARQL results into a Pandas DataFrame.
    Credit: https://lawlesst.github.io/notebook/sparql-dataframe.html
    """
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()

    cols = results['head']['vars']
    out = []
    for row in results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)


In [5]:
def getInfo(URI):
    return '''
select * where {
    <%s> dbo:abstract ?y.
    filter(lang(?y)="en")
 }'''%(URI)

In [6]:
def retrieveInfo(endpoint,uri) :
    query = getInfo(uri)
    result = sparql_to_dataframe(endpoint, query)
    if len(result)>0:
        result = result.iloc[0][0]
    else:
        result = "No more data"
    return result

In [8]:
df_train = pd.read_csv('data_train.csv')
df_train.head()

,text,label
0,Here are Thursday's biggest analyst calls: App...,0
1,Buy Las Vegas Sands as travel to Singapore bui...,0
2,"Piper Sandler downgrades DocuSign to sell, cit...",0
3,"Analysts react to Tesla's latest earnings, bre...",0
4,Netflix and its peers are set for a ‘return to...,0


In [43]:
df_test = pd.read_csv('data_test.csv')
df_test.head()

,text,label
0,Analyst call of the day for @CNBCPro subscribe...,0
1,"Loop upgrades CSX to buy, says it's a good pla...",0
2,BofA believes we're already in a recession — a...,0
3,JPMorgan sees these derivative plays as best w...,0
4,Morgan Stanley's Huberty sees Apple earnings m...,0


In [44]:
df_test = df_test.iloc[:2,0]
df_test.head()

0    Analyst call of the day for @CNBCPro subscribe...
1    Loop upgrades CSX to buy, says it's a good pla...
Name: text, dtype: object

In [50]:
def modifyText(text,index,modification):
    part1 = text[:index]
    part2 = text[index:]
    part1 +="["+modification+"] "
    part1 += part2
    return part1

def modify(tweet):
    modified = tweet
    annotations = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate',
    tweet,confidence=0.4,support = 20)
    URIs = getURIs(annotations)
    for k in range(len(annotations)):
        j = len(annotations)-k-1
        infos = retrieveInfo(endpoint,annotations[j]["URI"])
        index = len(annotations[j]["surfaceForm"]) + annotations[j]["offset"]
        modified = modifyText(modified,index,infos)
    return modified
        


In [51]:
df = pd.DataFrame()
for index, data in df_test.items():
    modified = modify(data)
    df = df.append({"original":data,"modified":modified}, ignore_index=True)

C:\Users\nicol\AppData\Local\Temp\ipykernel_5116\3338115639.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"original":data,"modified":modified}, ignore_index=True)
C:\Users\nicol\AppData\Local\Temp\ipykernel_5116\3338115639.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"original":data,"modified":modified}, ignore_index=True)


In [52]:
print(df.iloc[0,0])
print(df.iloc[0,1])

Analyst call of the day for @CNBCPro subscribers: Goldman Sachs reiterates Exxon as buy and says it’s bullish heading into earnings next week.  Subscribe to see the analyst calls of the day curated by CNBC Pro.  https://t.co/q2fXwZqMZ4
Analyst call of the day for @CNBCPro subscribers: Goldman Sachs[Goldman Sachs (/ˈsæks/) is an American multinational investment bank and financial services company. Founded in 1869, Goldman Sachs is headquartered at 200 West Street in Lower Manhattan, with regional headquarters in London, Warsaw, Bangalore, Hong Kong, Tokyo, Dallas and Salt Lake City, and additional offices in other international financial centers. Goldman Sachs is the second largest investment bank in the world by revenue and is ranked 57th on the Fortune 500 list of the largest United States corporations by total revenue. It is considered a systemically important financial institution by the Financial Stability Board. The company has been criticized for a lack of ethical standards, wor